XGBoost 의 Python 패키지는 pip install 로 설치가 가능합니다. 10k 데이터는 classifier comparison 튜토리얼에서 이용할 것이기 때문에 이번에는 24 만여 문장으로 이뤄진 데이터를 이용합니다.

```
pip install xgboost
```

In [1]:
import config
import numpy as np
import warnings
import xgboost as xgb
from navermovie_comments import load_sentiment_dataset

warnings.filterwarnings('ignore')
print(xgb.__version__)

texts, x, y, idx_to_vocab = load_sentiment_dataset(data_name='small', tokenize='komoran')
print(x.shape)
print(np.unique(y))

0.90
(239708, 10178)
[-1  1]


이 데이터의 label 은 negative 를 -1 로, positive 를 1 로 지정하였습니다만, xgboost 는 [0, 1] 의 class label 만 입력가능합니다. -1 인 부분을 찾아 모두 0 으로 바꿔줍니다.

In [2]:
# label used in xgboost must be [0, 1]
y[np.where(y == -1)[0]] = 0
np.unique(y)

array([0, 1])

학습 정확도와 테스트 정확도를 확인하기 위하여 24 만여 문장을 9:1 로 학습용과 테스트용으로 나눕니다.

In [3]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

XGBoost 의 입력데이터는 DMatrix 형식이며, data 에는 scipy.sparse.csr_matrix 와 같은 sparse matrix 및 numpy.ndarray 와 같은 dense matrix 도 입력할 수 있습니다. 학습용 데이터 `dtrain` 과 테스트용 데이터 `dtest` 를 나눠 만듭니다.

In [4]:
dtrain = xgb.DMatrix(data=x_train, label=y_train)
dtest = xgb.DMatrix(data=x_test)

모델 학습에 이용할 패러매터를 설정합니다. binary classification 이기 때문에 이 값도 지정합니다. xgboost 는 설정할 수 있는 패러매터가 많은데, 이들에 대해서는 official document 를 살펴보시기 바랍니다.

- https://xgboost.readthedocs.io/en/latest/parameter.html

In [5]:
param = {'max_depth': 4, 'eta': 0.3, 'silent': 0, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

`num_round` 는 앙상블 모델을 구성하는 개별 모델의 개수입니다. 모델의 학습이 끝나면 학습된 모델 객체가 return 됩니다.

In [6]:
num_round = 10
bst = xgb.train(param, dtrain, num_round)

학습된 모델은 저장할 수 있으며,

In [7]:
bst.save_model('xgb.model')

load_model 을 이용하여 불러들일 수도 있습니다.

In [8]:
loaded_bst = xgb.Booster({'nthread': 4})  # init model
loaded_bst.load_model('xgb.model')  # load data

정확도를 측정하는 함수는 여러 번 이용할 것이기 때문에 accuracy 라는 함수로 만듭니다. predict 함수는 positive class 에 속할 확률을 return 합니다. 이 값이 0.5 이상인 값들을 모두 1 로, 그렇지 않은 값을 0 으로 만들어 label 을 얻습니다.

학습 데이터의 정확도와 테스트 데이터의 정확도를 각각 구합니다.

In [9]:
def accuracy(bst, x, y):
    pred_score = bst.predict(x)
    y_pred = np.zeros(pred_score.shape[0])
    y_pred[np.where(pred_score > 0.5)[0]] = 1
    n_correct = np.where(y_pred == y)[0].shape[0]
    accuracy = n_correct / y.shape[0]
    return accuracy

print('train accuracy : {}'.format(accuracy(bst, dtrain, y_train)))
print('test  accuracy : {}'.format(accuracy(bst, dtest, y_test)))

train accuracy : 0.7784571028613544
test  accuracy : 0.7768970839764716


이번에는 10 개가 아닌, 1000 개의 모델을 이용하여 앙상블 모델을 구축합니다.

In [10]:
num_round = 1000
bst = xgb.train(param, dtrain, num_round)

print('train accuracy : {}'.format(accuracy(bst, dtrain, y_train)))
print('test  accuracy : {}'.format(accuracy(bst, dtest, y_test)))

train accuracy : 0.9050093400761112
test  accuracy : 0.8860289516499104


같은 데이터에 대하여 L1 regularization + logistic regression 을 학습한 결과 입니다. XGBoost 와 비슷한 성능을 확인할 수 있습니다.

```python
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


# for each cost
for cost in [100, 10, 1]:

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

    # build lasso regression
    logistic_regression = LogisticRegression(
        penalty='l1', C=cost)
    
    logistic_regression.fit(x_train, y_train)
    y_train_pred = logistic_regression.predict(x_train)
    train_acc = np.where(y_train_pred == y_train)[0].shape[0]
    train_acc /= y_train.shape[0]

    y_test_pred = logistic_regression.predict(x_test)
    test_acc = np.where(y_test_pred == y_test)[0].shape[0]
    test_acc /= y_test.shape[0]

    print('L1 lambda = {}, training accuracy = {}, test accoracy = {}'.format(1/cost, train_acc, test_acc))
```

```
L1 lambda = 0.01, training accuracy = 0.905139127734232, test accoracy = 0.8806057319260774
L1 lambda = 0.1, training accuracy = 0.905064963358163, test accoracy = 0.8820658295440323
L1 lambda = 1.0, training accuracy = 0.8995536231615346, test accoracy = 0.8841099662091694
```

`predict_proba` 함수는 XGBClassifier 클래스에 구현되어 있습니다. 이전에 학습한 모델 역시 objective 를 binary 로 설정하였기 때문에 classifier 입니다. 이를 로딩한 뒤, 학습 데이터에 대한 판별 확률과 테스트 데이터에 대한 판별 확률의 평균을 확인합니다.

In [27]:
classifier = xgb.XGBClassifier()
classifier.load_model('0001.model')

In [34]:
def check_average_prediction_probability(classifier, x):
    proba = classifier.predict_proba(x)
    mean_proba = proba.max(axis=1).mean()
    return mean_proba

print('average prection probability with train data = {}'.format(
    check_average_prediction_probability(classifier, x_train)))
print('average prection probability with test data = {}'.format(
    check_average_prediction_probability(classifier, x_test)))

average prection probability with train data = 0.7423478960990906
average prection probability with test data = 0.7415752410888672
